In [1]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.nn import init
from torch.nn.modules.utils import _triple
import math
from dataset import AlignedDataset
from networks import Encoder, Decoder
from losses import Neg_Pearson
from torch.utils.data import DataLoader
import os
import itertools
import numpy as np

In [2]:
train_file_list = "/mnt/hdd.user/datasets/FAS/Oulu-NPU/Protocols/Protocol_1/Train_proto.txt"
train_batch_size = 1
train_dataset = AlignedDataset(train_file_list) 
train_data_loader = DataLoader(train_dataset, batch_size=train_batch_size,
                                    shuffle = False,num_workers=1)

In [3]:
lossnp = Neg_Pearson()

In [4]:
encoder = Encoder()
decoder = Decoder()

In [5]:
print(encoder.__class__.__name__)
classname = encoder.__class__.__name__

Encoder


In [6]:
optimizer_G = torch.optim.Adam(itertools.chain(encoder.parameters(),decoder.parameters()), lr=0.00003,betas=(0.5, 0.999))

In [8]:
for i, data in enumerate(train_data_loader):
    print("index: {}".format(i))
    if i > 10:
        break
    if (data['A'].shape[2] == 64):
#         print(data['A'])
#         print(data['A'].shape)
#         print(data['B'].shape)
#         print(data['A_paths'])
#         print(data['B'])
        lantent = encoder(data['A'])
        sig = decoder(lantent)
#         print(lantent)
        print(sig)
        print(data['B'])
#         optimizer_G.zero_grad()
        loss_np = lossnp(sig, data['B'])
        loss_final = 0.1*loss_np
        print("loss np:{}, final:{}".format(loss_np, loss_final))
#         loss_final.backward()
#         optimizer_G.step()
    print("---------------------------------------")

index: 0
input shape: torch.Size([1, 3, 64, 128, 128])
x0.0: tensor([[[[[0.0178, 0.0178, 0.0178,  ..., 0.0178, 0.0178, 0.0178],
           [0.0178, 0.0178, 0.0178,  ..., 0.0178, 0.0178, 0.0178],
           [0.0178, 0.0178, 0.0178,  ..., 0.0178, 0.0178, 0.0178],
           ...,
           [0.0178, 0.0178, 0.0178,  ..., 0.0178, 0.0178, 0.0178],
           [0.0178, 0.0178, 0.0178,  ..., 0.0178, 0.0178, 0.0178],
           [0.0178, 0.0178, 0.0178,  ..., 0.0178, 0.0178, 0.0178]],

          [[0.0178, 0.0178, 0.0178,  ..., 0.0178, 0.0178, 0.0178],
           [0.0178, 0.0178, 0.0178,  ..., 0.0178, 0.0178, 0.0178],
           [0.0178, 0.0178, 0.0178,  ..., 0.0178, 0.0178, 0.0178],
           ...,
           [0.0178, 0.0178, 0.0178,  ..., 0.0178, 0.0178, 0.0178],
           [0.0178, 0.0178, 0.0178,  ..., 0.0178, 0.0178, 0.0178],
           [0.0178, 0.0178, 0.0178,  ..., 0.0178, 0.0178, 0.0178]],

          [[0.0178, 0.0178, 0.0178,  ..., 0.0178, 0.0178, 0.0178],
           [0.0178, 0.0178, 0.01

# test Pearson

In [13]:
def pearson(vector1, vector2):
    n = len(vector1)
    #simple sums
    sum1 = sum(float(vector1[i]) for i in range(n))
    sum2 = sum(float(vector2[i]) for i in range(n))
    #sum up the squares
    sum1_pow = sum([pow(v, 2.0) for v in vector1])
    sum2_pow = sum([pow(v, 2.0) for v in vector2])
    #sum up the products
    p_sum = sum([vector1[i]*vector2[i] for i in range(n)])
    #分子num，分母den
    num = p_sum - (sum1*sum2/n)
    den = math.sqrt((sum1_pow-pow(sum1, 2)/n)*(sum2_pow-pow(sum2, 2)/n))
    if den == 0:
        return 0.0
    return num/den


In [39]:
x = sig.squeeze(dim=0).detach().numpy()
y = data['B'].squeeze(dim=0).detach().numpy()

In [45]:
x = np.array([0, 1, 2, 3, 4, 5])
y = np.array([23,34,245,26,242,42])
p = pearson(x, y)

In [46]:
print(p)

0.2433875406934647


In [19]:
lll = torch.zeros(3, requires_grad=True)
print(lll)

tensor([0., 0., 0.], requires_grad=True)


In [20]:
xxx = torch.sum(lll)
print(xxx)

tensor(0., grad_fn=<SumBackward0>)
